# L5: Prompt Compression


<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

In [ ]:
# 警告控制
import warnings

# 忽略所有警告信息，以免干扰程序输出
warnings.filterwarnings('ignore')

In [ ]:
#pip install llmlingua

In [ ]:
import custom_utils  # 导入自定义工具库 custom_utils

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>utils</code> files:</b> To access <code>requirements.txt</code> for this notebook, 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Data Loading

In [ ]:
from datasets import load_dataset
import pandas as pd

# 加载数据集
dataset = load_dataset("MongoDB/airbnb_embeddings", streaming=True, split="train")
dataset = dataset.take(100)  # 取前100条数据

# 将数据集转换为 pandas 数据框
dataset_df = pd.DataFrame(dataset)

# 显示前5条数据
dataset_df.head(5)

In [ ]:
# 打印数据框的列名
print("Columns:", dataset_df.columns)

## Document Modelling

In [ ]:
# 使用自定义工具库处理记录，并将结果存储在 listings 变量中
listings = custom_utils.process_records(dataset_df)

## Database Creation and Connection

In [ ]:
# 使用自定义工具库连接到数据库，并获取数据库和集合对象
db, collection = custom_utils.connect_to_database()

In [ ]:
# 删除集合中所有现有的记录
collection.delete_many({})

## Data Ingestion

In [ ]:
# 插入处理后的记录到集合中
collection.insert_many(listings)
print("Data ingestion into MongoDB completed")

## Vector Search Index defintion

In [ ]:
# 创建带有过滤器的向量搜索索引
custom_utils.setup_vector_search_index_with_filter(collection=collection)


<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note:</b> If the output of the previous cell is <code>Error creating vector search index: Duplicate Index</code> you may proceed to the next cell if you intend to still use a previously created index.</p>

## Handling User Query

In [ ]:
from pydantic import BaseModel
from typing import Optional
import custom_utils

class SearchResultItem(BaseModel):
    name: str  # 房源名称
    accommodates: Optional[int] = None  # 可容纳人数，可选
    address: custom_utils.Address  # 地址信息
    neighborhood_overview: Optional[str] = None  # 邻里概况，可选
    notes: Optional[str] = None  # 备注，可选
    averageReviewScore: Optional[float] = None  # 平均评论得分，可选
    number_of_reviews: Optional[float] = None  # 评论数量，可选
    combinedScore: Optional[float] = None  # 综合评分，可选

## Boosting Search Results After Vector Search

In [ ]:
# 定义计算平均评论得分和评论数量加权得分的聚合阶段
review_average_stage = {
    "$addFields": {
        "averageReviewScore": {
            "$divide": [
                {
                    "$add": [
                        "$review_scores.review_scores_accuracy",
                        "$review_scores.review_scores_cleanliness",
                        "$review_scores.review_scores_checkin",
                        "$review_scores.review_scores_communication",
                        "$review_scores.review_scores_location",
                        "$review_scores.review_scores_value",
                    ]
                },
                6  # 除以评论评分类型的数量以获得平均值
            ]
        },
        # 根据评论数量计算评分提升因子
        "reviewCountBoost": "$number_of_reviews"
    }
}

In [ ]:
# 定义加权评分的聚合阶段
weighting_stage = {
    "$addFields": {
        "combinedScore": {
            # 结合平均评论得分和评论数量提升的示例公式
            "$add": [
                {"$multiply": ["$averageReviewScore", 0.3]},  # 加权平均评论得分
                {"$multiply": ["$reviewCountBoost", 0.7]}   # 加权评论数量提升
            ]
        }
    }
}

In [ ]:
# 应用 combinedScore 进行排序的聚合阶段
sorting_stage_sort = {
    "$sort": {"combinedScore": -1}  # 按降序排列，以提升较高的综合评分
}

In [ ]:
# 定义额外的聚合阶段，包括计算平均评论得分、加权评分和排序
additional_stages = [review_average_stage, weighting_stage, sorting_stage_sort]

## Modified Handling User Query

In [ ]:
from IPython.display import display, HTML
import pprint

def handle_user_query(query, db, collection, stages=[], vector_index="vector_index_text"):
    """
    处理用户查询并返回系统响应和源信息。

    Args:
    query (str): 用户的查询字符串。
    db (MongoClient.database): 数据库对象。
    collection (MongoCollection): 要搜索的 MongoDB 集合。
    stages (list): 额外的聚合阶段要包括在管道中。
    vector_index (str): 向量索引名称，默认为 "vector_index_text"。

    Returns:
    str: 系统响应。
    """
    # 执行向量搜索
    get_knowledge = custom_utils.vector_search_with_filter(query, db, collection, stages, vector_index)

    # 检查是否有结果
    if not get_knowledge:
        return "No results found.", "No source information available."
    
    # 将搜索结果转换为 SearchResultItem 模型列表
    search_results_models = [
        SearchResultItem(**result)
        for result in get_knowledge
    ]

    # 将搜索结果转换为 DataFrame 以便在 Jupyter 中更好地呈现
    search_results_df = pd.DataFrame([item.dict() for item in search_results_models])

    # 打印未压缩的提示（查询信息）
    print("Uncompressed Prompt (Query Info):\n")
    print(search_results_df)

    # 使用 OpenAI 的 completion 生成系统响应
    completion = custom_utils.openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system", 
                "content": "You are an Airbnb listing recommendation system."
            },
            {
                "role": "user", 
                "content": f"Answer this user query: {query} with the following context:\n{search_results_df}"
            }
        ]
    )
    system_response = completion.choices[0].message['content']

    # 打印用户问题、系统响应和源信息
    print(f"- User Question:\n{query}\n")
    print(f"- System Response:\n{system_response}\n")

    # 以 HTML 表格形式显示 DataFrame
    display(HTML(search_results_df.to_html()))

    # 返回结构化响应和源信息作为字符串
    return system_response

In [ ]:
query = """
I want to stay in a place that's warm and friendly, 
and not too far from restaurants, can you recommend a place? 
Include a reason as to why you've chosen your selection.
"""

# 处理用户查询并获取响应
response = handle_user_query(
    query, 
    db, 
    collection, 
    additional_stages, 
    vector_index="vector_index_with_filter"
)
response

## Prompt Compression

In [ ]:
import json
from llmlingua import PromptCompressor

# 初始化 PromptCompressor 实例
llm_lingua = PromptCompressor(
    model_name="microsoft/llmlingua-2-bert-base-multilingual-cased-meetingbank",
    model_config={"revision": "main"},
    use_llmlingua2=True,
    device_map="cpu",
)

# 定义压缩查询提示的函数
def compress_query_prompt(query):
    """
    压缩用户查询提示。

    Args:
    query (dict): 包含 demonstration_str, instruction 和 question 的字典。

    Returns:
    str: 压缩后的查询提示，以 JSON 字符串形式返回。
    """
    demonstration_str = query['demonstration_str']
    instruction = query['instruction']
    question = query['question']

    # 执行 6 倍压缩
    compressed_prompt = llm_lingua.compress_prompt(
        demonstration_str.split("\n"), 
        instruction=instruction,
        question=question,
        target_token=500,
        rank_method="longllmlingua", 
        context_budget="+100",
        dynamic_context_compression_ratio=0.4,
        reorder_context="sort",
    )

    return json.dumps(compressed_prompt, indent=4)


In [ ]:
def handle_user_query_with_compression(query, db, collection, stages=[], vector_index="vector_index_text"):
    """
    处理用户查询并压缩提示。

    Args:
    query (str): 用户的查询字符串。
    db (MongoClient.database): 数据库对象。
    collection (MongoCollection): 要搜索的 MongoDB 集合。
    stages (list): 额外的聚合阶段要包括在管道中。
    vector_index (str): 向量索引名称，默认为 "vector_index_text"。

    Returns:
    tuple: 搜索结果的 DataFrame 和压缩后的提示。
    """
    # 执行向量搜索以从数据库获取知识
    get_knowledge = custom_utils.vector_search_with_filter(query, db, collection, stages, vector_index)

    # 检查是否有结果
    if not get_knowledge:
        return None, "No results found."

    # 将搜索结果转换为 SearchResultItem 模型列表
    search_results_models = [SearchResultItem(**result) for result in get_knowledge]

    # 将搜索结果转换为 DataFrame 以便更好地呈现
    search_results_df = pd.DataFrame([item.dict() for item in search_results_models])

    # 准备压缩信息
    query_info = {
        'demonstration_str': search_results_df.to_string(),  # 信息检索过程中的结果
        'instruction': "Write a high-quality answer for the given question using only the provided search results.",
        'question': query  # 用户查询
    }

    # 使用预定义函数压缩查询提示
    compressed_prompt = compress_query_prompt(query_info)

    # 可选：打印压缩后的提示以进行调试
    print("Compressed Prompt:\n")
    pprint.pprint(compressed_prompt)
    print("\n" + "=" * 80 + "\n")

    return search_results_df, compressed_prompt

In [ ]:
def handle_system_response(query, compressed_prompt):
    """
    处理系统响应。

    Args:
    query (str): 用户的查询字符串。
    compressed_prompt (str): 压缩后的提示信息。

    Returns:
    str: 系统响应。
    """
    # 使用 OpenAI 的 completion 生成系统响应
    completion = custom_utils.openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are an Airbnb listing recommendation system."
            },
            {
                "role": "user",
                "content": f"Answer this user query: {query} with the following context:\n{compressed_prompt}"
            }
        ]
    )

    system_response = completion.choices[0].message['content']

    # 打印用户问题和系统响应
    print(f"- User Question:\n{query}\n")
    print(f"- System Response:\n{system_response}\n")

    # 返回系统响应
    return system_response

In [ ]:
# 压缩查询并获取搜索结果
results, compressed_prompt = handle_user_query_with_compression(
    query, 
    db, 
    collection, 
    additional_stages, 
    vector_index="vector_index_with_filter"
)


In [ ]:
if compressed_prompt:
    # 使用压缩后的提示处理系统响应
    system_response = handle_system_response(query, compressed_prompt)
else:
    print("No valid results to display.")

## Additional Resouces

- [The MongoDB Developer Center for tutorials, articles and videos](https://mdb.link/developer_center)

- [The GenAI Showcase Repo for code showcasing building AI applications and demo](https://github.com/mongodb-developer/GenAI-Showcase)

- [DeepLearning AI forum for question in regards to this course](https://community.deeplearning.ai/)